In [1]:
# %%
import numpy as np
import sapien.core as sapien
from PIL import Image
import json
from datagen.data_utils import *
from pathlib import Path as P
from utils.visualization import overlay_images
# from utils.viewpoint import calculate_E2 as CE2
# from utils.viewpoint import conversion_matrix_axis as cma
from utils.viewpoint import * 
# from utils.viewpoint import conversion_matrix
# %%
# def setup_scene(urdf):
#     # Set up the SAPIEN scene and camera
#     camera, asset, scene = scene_setup(urdf_file=urdf)
#     return camera, asset, scene
# conversion_matrix = np.array([
#     [0, 0, -1],
#     [-1, 0, 0],
#     [0, 1, 0]
# # ])
# def normalize(v):
#     return v / np.sqrt(np.sum(v**2))

# def calculate_E2(E1, axis_position, axis_direction, angle_degrees):
#     # Convert the angle from degrees to radians
#     angle_radians = degrees_to_radians(angle_degrees)
    
#     # Create a 3x3 rotation matrix around the axis
#     R = get_rotation_axis_angle(axis_direction, angle_radians)
#     # Create a 4x4 transformation matrix for the rotation
#     rotation_matrix = np.eye(4)
#     rotation_matrix[:3, :3] = R
#     print(R)
    
#     # Create a 4x4 transformation matrix for translation to the axis position
#     translation_matrix = np.eye(4)
#     translation_matrix[:3, 3] = axis_position
#     # Calculate E2 by combining translation and rotation with E1
#     # E2 = np.dot(rotation_matrix, np.dot(rotation_matrix, E1))
#     E2 = change_apply_change_basis(E1, rotation_matrix, translation_matrix)
#     return E2

# def change_apply_change_basis(A, T, P):
#     """
#     Perform change of basis, apply transformation, and change back to the original basis.

#     Args:
#     A (numpy.ndarray): The original matrix in the B1 basis.
#     T (numpy.ndarray): The transformation matrix to be applied.
#     P (numpy.ndarray): The change of basis matrix from B1 to B2.

#     Returns:
#     numpy.ndarray: The resulting matrix after the entire process, expressed in the B1 basis.
#     """
#     # Step 1: Change A to the B2 basis
#     A_B2 = np.linalg.inv(P).dot(A).dot(P)
    
#     # Step 2: Apply the transformation T in the B2 basis
#     A_B2_transformed = T.dot(A_B2)
    
#     # Step 3: Change A_B2_transformed back to the original B1 basis
#     A_B1_transformed = P.dot(A_B2_transformed).dot(np.linalg.inv(P))
    
#     return A_B1_transformed

# def get_rotation_axis_angle(k, theta):
#     '''
#     Rodrigues' rotation formula
#     args:
#     * k: direction vector of the axis to rotate about
#     * theta: the (radian) angle to rotate with
#     return:
#     * 3x3 rotation matrix
#     '''
#     k = normalize(k)
#     kx, ky, kz = k[0], k[1], k[2]
#     cos, sin = np.cos(theta), np.sin(theta)
#     R = np.zeros((3, 3))
#     R[0, 0] = cos + (kx**2) * (1 - cos)
#     R[0, 1] = kx * ky * (1 - cos) - kz * sin
#     R[0, 2] = kx * kz * (1 - cos) + ky * sin
#     R[1, 0] = kx * ky * (1 - cos) + kz * sin
#     R[1, 1] = cos + (ky**2) * (1 - cos)
#     R[1, 2] = ky * kz * (1 - cos) - kx * sin
#     R[2, 0] = kx * kz * (1 - cos) - ky * sin
#     R[2, 1] = ky * kz * (1 - cos) + kx * sin
#     R[2, 2] = cos + (kz**2) * (1 - cos)
#     return R


# def transform_V1_to_V2(joint_state, V1, delta_rotation):
#     try:
#         # Extract the rotation matrix R_joint from the joint state
#         R_joint = joint_state[:3, :3]

#         u = R_joint[0, 2]
#         v = R_joint[1, 2]
#         w = R_joint[2, 2]


#         # Calculate the rotation matrix R_theta for the delta rotation along the joint axis
#         cos_theta = np.cos(delta_rotation)
#         sin_theta = np.sin(delta_rotation)
#         R_theta = np.array([[cos_theta + u**2 * (1 - cos_theta), u * v * (1 - cos_theta) - w * sin_theta, u * w * (1 - cos_theta) + v * sin_theta],
#                             [v * u * (1 - cos_theta) + w * sin_theta, cos_theta + v**2 * (1 - cos_theta), v * w * (1 - cos_theta) - u * sin_theta],
#                             [w * u * (1 - cos_theta) - v * sin_theta, w * v * (1 - cos_theta) + u * sin_theta, cos_theta + w**2 * (1 - cos_theta)]])


#         # Create 4x4 transformation matrices
#         T_joint = np.eye(4)
#         T_joint[:3, :3] = R_joint

#         T_theta = np.eye(4)
#         T_theta[:3, :3] = R_theta

#         # Compute V2 by multiplying the transformation matrices
#         # V2 = np.dot(np.dot(np.dot(T_joint, T_theta), np.linalg.inv(T_joint)), V1)
#         V2_prime = np.dot(np.linalg.inv(T_joint), np.dot(T_theta, V1))
#         V2 = np.dot(V2_prime, T_joint)

#         return V2

#     except Exception as e:
#         print(f"Error calculating V2: {str(e)}")
#         return None
# def degrees_to_radians(degrees):
#     radians = degrees * (math.pi / 180)
#     return radians
def render_image(camera, scene, save_path):
    # Render an image using the provided camera and save it to the specified path
    scene.step()
    scene.update_render()
    camera.take_picture()
    rgba = camera.get_float_texture('Color')
    rgba_img = (rgba * 255).clip(0, 255).astype("uint8")
    

    seg_labels = camera.get_uint32_texture('Segmentation')  # [H, W, 4]
    mask = seg_labels.sum(axis=-1)
    mask[mask>0] = 1
    rgba_img[:, :, -1] = rgba_img[:, :, -1] * mask
    rgba_pil = Image.fromarray(rgba_img, 'RGBA')
    rgba_pil.save(save_path)

def load_json_to_dict(fname):
    # Load JSON data from a file into a dictionary
    try:
        with open(fname, 'r') as file:
            data = json.load(file)
        return data
    except Exception as e:
        print(f"Error loading JSON from {fname}: {str(e)}")
        return None
    

urdf = 'data_base/laptop/10211/mobility.urdf'
# Load JSON data
json_fname = './data_base/laptop/10211/mobility_v2.json'
meta = load_json_to_dict(json_fname)
for link in meta:
    if link['joint'] == 'hinge':
        print(link['id'])
        print(link['jointData'])
        origin =  link['jointData']['axis']['origin']
        direction =  link['jointData']['axis']['direction']
# Setup the scene
camera, asset, scene = scene_setup(urdf)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
3
{'axis': {'origin': [0, -0.007706040424053753, -0.24714714808389615], 'direction': [1, 0, 0]}, 'limit': {'a': 105.11999999999999, 'b': -27, 'noLimit': False}}


In [2]:
# %%
# point = random_point_in_sphere(5, theta_range=[-0.01 * math.pi, 0.01*math.pi], phi_range=[0.01*math.pi, 0.01*math.pi])

def point_in_sphere(r, theta, phi):
    x = r * math.sin(phi) * math.cos(theta)
    y = r * math.sin(phi) * math.sin(theta)
    z = r * math.cos(phi)
    
    return x, y, z

point = point_in_sphere(5, 1.2*math.pi, degrees_to_radians(60))
mat44 = calculate_cam_ext(point)
print("point")
print(point)
print('mat44:')
print(mat44)
# print(point)
# mat44 = np.eye(4)
# mat44[0, -1] = 4
# mat44[1, -1] = 2
# mat44[2, -1] = 1
camera.set_pose(sapien.Pose.from_transformation_matrix(mat44))
# Render image with default pose
render_image(camera, scene, './draft/test_view.png')
seg_labels = camera.get_uint32_texture('Segmentation')  # [H, W, 4]
seg_view = seg_labels[..., 1].astype(np.uint8)  # actor-level
seg_view[seg_view != 2] = 0
seg_pil = Image.fromarray(seg_view*255)
seg_pil.save('./draft/test_view_seg.png')
view_0 = camera.get_model_matrix()
art_degree = radians_to_degree(asset.get_qlimits()[0, 1])

# Rotate the joint and render a new image
asset.set_qpos(degrees_to_radians(art_degree))
render_image(camera, scene, './draft/test_view_15.png')
seg_labels = camera.get_uint32_texture('Segmentation')  # [H, W, 4]
seg_view_15 = seg_labels[..., 1].astype(np.uint8)  # actor-level
seg_view_15[seg_view_15 != 1] = 0
seg_view_15 = Image.fromarray(seg_view_15*255)
seg_view_15.save('./draft/test_view_15_seg.png')
# Calculate joint state and transformation
joint_state = asset.get_links()[-1].pose.to_transformation_matrix()
# v2 = transform_V1_to_V2(joint_state, mat44, degrees_to_radians(15))


# dirs, ori = np.dot(conversion_matrix, direction), np.dot(conversion_matrix, origin)


# v2 = calculate_E2(mat44, ori, dirs, art_degree)
v2 = calculate_E2(mat44, origin, direction, art_degree)
# view_2 = calculate_E2(view_0, np.array(origin), np.array(direction), art_degree)
# Set the new joint configuration and render the image
asset.set_qpos(0)
camera.set_pose(sapien.Pose.from_transformation_matrix(v2))
render_image(camera, scene, './draft/test_view_v2.png')
seg_labels = camera.get_uint32_texture('Segmentation')  # [H, W, 4]
seg_view_15_v2 = seg_labels[..., 1].astype(np.uint8)  # actor-level
seg_view_15_v2[seg_view_15_v2 != 3] = 0
seg_view_15_v2_pil = Image.fromarray(seg_view_15_v2*255)
seg_view_15_v2_pil.save('./draft/test_view_v2_seg.png')

view_1 = camera.get_model_matrix()


inerf_json = {
    'view_0': view_0.tolist(),
    'view_1': view_1.tolist()
}

with open('./draft/inerf_pose.json', 'w') as f:
    json.dump(inerf_json, f)


# Overlay images
rgba_15_pil = Image.open('./draft/test_view_15.png')
rgba_v2_pil = Image.open('./draft/test_view_v2.png')
overlayed = overlay_images(rgba_15_pil, rgba_v2_pil, 0.5)
overlayed.save('./draft/overlayed_view.png')


rgba_pil = Image.open('./draft/test_view.png')
overlayed = overlay_images(rgba_pil, rgba_15_pil, 0.5)
overlayed.save('./draft/overlayed_art.png')

overlayed = overlay_images(rgba_pil, rgba_v2_pil, 0.5)
overlayed.save('./draft/overlayed_base.png')

point
(-3.503146346110184, -2.545184802275635, 2.5000000000000004)
mat44:
[[ 0.70062927 -0.58778525  0.4045085  -3.50314635]
 [ 0.50903696  0.80901699  0.29389263 -2.5451848 ]
 [-0.5         0.          0.8660254   2.5       ]
 [ 0.          0.          0.          1.        ]]
[[ 0.89100652 -0.         -0.45399051]
 [ 0.          1.         -0.        ]
 [ 0.45399051  0.          0.89100652]]
Pose([0, 0, 0], [0.97237, 0, -0.233445, 0])
P
 [[ 1.          0.          0.          0.24714715]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.         -0.00770604]
 [ 0.          0.          0.          1.        ]]
t
 [[ 0.70062927 -0.58778525  0.4045085  -3.75029349]
 [ 0.50903696  0.80901699  0.29389263 -2.5451848 ]
 [-0.5         0.          0.8660254   2.50770604]
 [ 0.          0.          0.          1.        ]]
A_B2
 [[ 0.70062927 -0.58778525  0.4045085  -3.58025213]
 [ 0.50903696  0.80901699  0.29389263 -2.42164252]
 [-0.5         0.          0.866025

In [3]:
# verify view0, view1
from utils.rotation import R_from_quaternions
view_0_torch = torch.Tensor(view_0)
view_1_torch = torch.Tensor(view_1)
origin_torch = torch.Tensor(origin)
Q = torch.Tensor([0.97237, 0, -0.233445, 0])
c2w = view_0_torch
E1 = view2pose_torch(c2w)
translation_matrix = torch.eye(4).to(c2w)
translation_matrix[:3, 3] = convert_ori_torch(origin_torch).view([3])
rotation_matrix = torch.eye(4).to(c2w)
R = R_from_quaternions(Q)
rotation_matrix[:3, :3] = R
E2 = change_apply_change_basis_torch(E1, rotation_matrix, translation_matrix)
view_1_test = pose2view_torch(E2)
print(view_1_test)
print(view_1_torch)
print(view_1_test - view_1_torch)

P
 tensor([[ 1.0000,  0.0000,  0.0000,  0.2471],
        [ 0.0000,  1.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  1.0000, -0.0077],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
t
 tensor([[ 7.0063e-01, -5.8779e-01,  4.0451e-01, -3.7503e+00],
        [ 5.0904e-01,  8.0902e-01,  2.9389e-01, -2.5452e+00],
        [-5.0000e-01,  2.9802e-08,  8.6603e-01,  2.5077e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])
A_B2
 tensor([[ 7.0063e-01, -5.8779e-01,  4.0451e-01, -3.5803e+00],
        [ 5.0904e-01,  8.0902e-01,  2.9389e-01, -2.4216e+00],
        [-5.0000e-01,  2.9802e-08,  8.6603e-01,  2.3775e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00]])
A_B2_transformed
 tensor([[ 0.8513, -0.5237, -0.0327, -4.2694],
        [ 0.5090,  0.8090,  0.2939, -2.4216],
        [-0.1274, -0.2668,  0.9553,  0.4929],
        [ 0.0000,  0.0000,  0.0000,  1.0000]])
tensor([[ 0.5237, -0.0327, -0.8513, -4.2329],
        [-0.8090,  0.2939, -0.5090, -2.5452],
        [ 

In [6]:
print(mat44)

[[ 0.70062927 -0.58778525  0.4045085  -3.50314635]
 [ 0.50903696  0.80901699  0.29389263 -2.5451848 ]
 [-0.5         0.          0.8660254   2.5       ]
 [ 0.          0.          0.          1.        ]]


In [7]:
print(v2)

[[ 0.8512605  -0.52372049 -0.03274761 -4.23286355]
 [ 0.50903696  0.80901699  0.29389263 -2.5451848 ]
 [-0.12742422 -0.26684893  0.9552773   0.52407873]
 [ 0.          0.          0.          1.        ]]


In [10]:
num_pose_img = 10
dof = asset.dof
qlimits = asset.get_qlimits()
qrange = qlimits[:, 1] - qlimits[:, 0]
qpos_list = np.random.randn(num_pose_img, dof)
qpos_list = qpos_list * qrange - qlimits[:,0]


In [31]:
dof = asset.dof
asset.set_qpos([np.inf] * dof)
qpos = asset.get_qpos()

In [32]:
qpos

array([0.4712389], dtype=float32)

In [13]:
ori, dirs, pos = get_local_art_GT(origin, direction, qpos, is_degree=False)

[[ 0.89100653 -0.         -0.45399052]
 [ 0.          1.         -0.        ]
 [ 0.45399052  0.          0.89100653]]


In [14]:
print(ori)
print(dirs)
print(pos)

[ 0.24714715  0.         -0.00770604]
[ 0 -1  0]
Pose([0, 0, 0], [0.97237, 0, -0.233445, 0])


# transform test with numpy

In [8]:
diff = view_0 - pose2view(mat44)
diff[abs(diff) < 1e-7] = 0
print(diff)

diff = mat44 - view2pose(pose2view(mat44))
diff[abs(diff) < 1e-7] = 0
print(diff)

[[ 1.10904854e-07  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 1.11001687e-07  0.00000000e+00 -1.16971783e-07  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


## transform test with torch

In [16]:
import torch

mat44_torch = torch.Tensor(mat44)
view_torch = torch.Tensor(view_0)
pose = sapien.Pose.from_transformation_matrix(mat44)
pose.q


array([ 0.91865003, -0.07997948,  0.24615154,  0.2984875 ], dtype=float32)

In [17]:
diff = view_torch - pose2view_torch(mat44_torch)
diff[abs(diff) < 1e-7] = 0
print(diff)

diff = mat44_torch - view2pose_torch(pose2view_torch(mat44_torch))
diff[abs(diff) < 1e-7] = 0
print(diff)

tensor([[ 1.1921e-07,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.1921e-07,  0.0000e+00, -1.1921e-07,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00]])
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])


In [18]:
ori, dirs = convert_ori_dir(origin, direction)

# Convert the angle from degrees to radians
angle_radians = degrees_to_radians(art_degree)

# Create a 3x3 rotation matrix around the axis
R = get_rotation_axis_angle(dirs, angle_radians)

from utils.rotation import *
Q = matrix_to_quaternion(torch.Tensor(R))

In [19]:
Q

tensor([ 0.9724,  0.0000, -0.2334,  0.0000])

In [20]:
test_mat44 = np.eye(4)
test_mat44[:3, :3] = R
test_pose = sapien.Pose.from_transformation_matrix(test_mat44)
test_pose.q

array([ 0.9723699 ,  0.        , -0.23344538,  0.        ], dtype=float32)

In [21]:
ori

array([ 0.24714715,  0.        , -0.00770604])

In [22]:
joints = asset.get_joints()

In [23]:
for joint in joints:
    print(joint)

In [24]:
joint.get_pose_in_parent()

Pose([0, -0.00770604, -0.247147], [-0, 0, 1, 0])

In [25]:
asset.set_qpos(np.array([0.2]))

In [26]:
asset.get_qpos()

array([0.2], dtype=float32)

In [27]:
asset.get_joints()[-1].get_pose_in_child()

Pose([0, 0, 0], [-0, 0, 1, 0])

In [28]:
link = joint.get_child_link()

In [29]:
link.pose

Pose([0.247147, 0, -0.00770605], [0.5, 0.5, -0.5, -0.5])